## Before you begin
You will need to install scikit-image, and restart jupyter notebook. Use the following commands:
```
# install sci-kit image
pip3 install scikit-image

# shut down the currently running Jupyter notebook server
# you can also just hit Ctrl-C in the terminal where you typed "jupyter notebook" to start the server
kill -9 $(ps aux | grep jupyter | grep Library | awk '{print $2}')

# restart the server
jupyter notebook
```

In [ ]:
from skimage import io
import numpy as np
import matplotlib.pyplot as plt

# This code is to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

First, let's define a function that lets you load images from disk. Note that these images must be in the same directory as this Jupyter notebook file.

In [ ]:
def load(image_path):
    """Loads an image from a file path, returning a numpy array of shape(image_height, image_width, 3).
    """
    out = io.imread(image_path)

    # Convert the image to be in the range (0, 1)
    out = out.astype(np.float64) / 255
    return out

Next, we'll load two sample images and test that we can display them.

In [ ]:
# You can change these paths to point to your own images if you want to try them out for fun.
# If you do, you must include these image files in your submission.
image1_path = './image1.jpg'
image2_path = './image2.jpg'

def display(img):
    # Show image
    plt.figure(figsize = (5,5))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
image1 = load(image1_path)
image2 = load(image2_path)

display(image1)
display(image2)

## Exercise 1: dimming

Recall from lecture that the value of each pixel represents its brightness. Implement a function that takes in an image and returns the same image, but half as bright as the original image.

In [ ]:
def dim_image(img):
    '''img is an image, represented as a 2D numpy array.'''
    # YOUR CODE GOES HERE
    out = img
    # END YOUR CODE
    return out
    
dim_image2 = dim_image(image2)
display (image2)
display (dim_image2)

## Exercise 2: convert to gray scale

Implement a function that takes in an RGB color image and outputs that same image in grayscale.

Hint: You may want to look at the documentation for skimage.color to see if there is something useful there.

In [ ]:
from skimage import color

def convert_to_grayscale(img):
    '''img is an RGB image, represented as a 2D numpy array.'''
    # YOUR CODE GOES HERE
    out = img
    # END YOUR CODE
    return out
    
grey_image = convert_to_grayscale(image1)
display(grey_image)

## Exercise 3: RGB exclusion

In the video we watched, we saw that you can represent a color image with a 2-dimensional array of pixels, where the R, G, and B pixels are grouped next to each other, and that this is how LCD screens are implemented.

In numpy, the representation is slightly different -- instead of having one 2-dimensional array, we have 3, and each one represents the brightness of the R, G, and B channels.

Implement a function that takes in an image and a channel, and returns an image without the specified channel.

In [ ]:
def rgb_exclusion(image, channel):
    """Return image **excluding** the rgb channel specified

    Args:
        image: numpy array of shape(image_height, image_width, 3).
        channel: str specifying the channel. Can be either "R", "G" or "B".

    Returns:
        out: numpy array of shape(image_height, image_width, 3).
    """
    ### YOUR CODE GOES HERE
    out = image
    ### END YOUR CODE

    return out

without_red = rgb_exclusion(image1, 'R')
without_blue = rgb_exclusion(image1, 'B')
without_green = rgb_exclusion(image1, 'G')

print("Below is the image without the red channel.")
display(without_red)

print("Below is the image without the green channel.")
display(without_green)

print("Below is the image without the blue channel.")
display(without_blue)

## Advanced exercise #1: LAB color space

RGB is not the only way that we can represent colors. In this advanced exercise, we'll explore the LAB and HSV color spaces.

Implement a function that takes in an image and an LAB channel, and returns an image without the specified channel. Again, you may want to look at the skimage.color module to see if there's something in there that can help you.

In [ ]:
image_l = lab_decomposition(image1, 'L')
image_a = lab_decomposition(image1, 'A')
image_b = lab_decomposition(image1, 'B')


print("Below is the image with only the L channel.")
display(image_l)

print("Below is the image with only the A channel.")
display(image_a)

print("Below is the image with only the B channel.")
display(image_b)

## Advanced exercise 1a

Explain what the L, A and B channels are and what happens when you take away the L and A channels.


## Advanced exercise 2: HSV exploration

Explain what the H, S and V channels are and what happens when you take away the both the H and S channels.

It may help you to implement a function that performs HSV decomposition and removes these channels; this is optional.

## Advanced exercise 3: combining images

Implement the following method, which takes in two images and returns a new image where the left half of the image is the left half of image1 and the right half of the image is the right half of image2. Exclude the specified channel for the given image. 

You should see the left half of the monkey without the red channel and the right half of the house image with no green channel.

In [ ]:
def mix_images(image1, image2, channel1, channel2):
    """
    Args:
        image1: numpy array of shape(image_height, image_width, 3).
        image2: numpy array of shape(image_height, image_width, 3).
        channel1: str specifying channel used for image1.
        channel2: str specifying channel used for image2.

    Returns:
        out: numpy array of shape(image_height, image_width, 3).
    """
    ### YOUR CODE GOES HERE
    out = image1
    ### END YOUR CODE

    return out

image_mixed = mix_images(image1, image2, channel1='R', channel2='G')
display(image_mixed)


## Advanced exercise 4: exploration

Implement a function that takes a single image, and performs a different operation to each of the 4 quadrants of the image, returning an image that merges the 4 quadrants back together.

For example, you might remove a channel or two from one quadrant, dim or brighten another quadrant, etc.